# SETUP

In [1]:
%%capture
!pip install fastapi kaleido python-multipart uvicorn
!pip install --upgrade datasets
!pip install pandas pydub
!pip install librosa scipy soundfile

In [2]:
from datasets import load_dataset
import numpy as np
import pandas as pd
from pydub import AudioSegment
import wave
import os
import io
import librosa
from scipy.signal import resample as scipy_resample

In [3]:
path = r"C:\Users\maron\OneDrive\02-Documents\03.PROJETS\00.INFORMATIQUE\02.AI\WOLOF\SPEECH_TO_TEXT"
os.chdir(path)

# Functions

In [4]:
def resample_audio(input_path, path_audio,  target_sr=16000):
    # Load the audio file
    audio, sr = librosa.load(path_audio + input_path + '.wav', sr=None)

    # Resample to 16 kHz
    #audio_resampled = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)

    # Calculate the duration of the original audio
    audio_duration = librosa.get_duration(y=audio, sr=sr)

    del audio
    del sr

    return audio_duration

# Data loading

In [5]:
separator = '  '
column_names = ['filename', 'transcript']
df_alffa_git_train = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\train\\text.csv", sep=separator, names=column_names)
df_alffa_git_train.head(4)

C:\Users\maron\AppData\Local\Temp\ipykernel_11876\3838009720.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_alffa_git_train = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\train\\text.csv", sep=separator, names=column_names)


,filename,transcript
0,WOL_01_lect_0001,sunu meetar ñówul tànkam dafa faxaj
1,WOL_01_lect_0002,Nanga ma ko fàttali lu ko moy dinaa ko fàtte
2,WOL_01_lect_0003,am na ñax mu nu daan foye xuusmaañàpp
3,WOL_01_lect_0004,maamam a daan tëgg laxaab yi ca dëkk ba


In [6]:
df_alffa_git_test = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\test\\text.csv", sep=separator, names=column_names)
df_alffa_git_test.head(4)

C:\Users\maron\AppData\Local\Temp\ipykernel_11876\3534181800.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_alffa_git_test = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\test\\text.csv", sep=separator, names=column_names)


,filename,transcript
0,WOL_05_lect_0005,tari boo xam ne dañu lay gàgganti rekk mokkul
1,WOL_05_lect_0009,dafa am jamano yoo xam ni xasaw dafay ànd ak x...
2,WOL_05_lect_0016,bu ngeen bëggee laale demleen ca àll ba
3,WOL_05_lect_0021,yenn saa damay fàtte ne nekkuma senegaal


In [7]:
df_alffa_git_dev = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\dev\\text.csv", sep=separator, names=column_names)
df_alffa_git_dev.head(4)

C:\Users\maron\AppData\Local\Temp\ipykernel_11876\3988973998.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_alffa_git_dev = pd.read_csv("DATA\BRUT\DATASET\\alffa_git\\dev\\text.csv", sep=separator, names=column_names)


,filename,transcript
0,WOL_03_lect_0001,seen suuf dafa nangu lu ñu fa ji mu sax
1,WOL_03_lect_0003,war na joxe kayit yii yépp bala weer wu dee
2,WOL_03_lect_0004,ndaxam mu ngi ne cocc ak téeméeri atam
3,WOL_03_lect_0006,boo fa demee dangay gis ndox miy ball


In [8]:
# Assuming dataset_waxal_df_train, dataset_waxal_df_validation, dataset_waxal_df_test, dataset_waxal_2_df_test are your DataFrames
dataframes = [df_alffa_git_test, df_alffa_git_train, df_alffa_git_dev]

df_alffa_git = pd.concat(dataframes, axis=0, ignore_index=True)
df_alffa_git.head(3)

,filename,transcript
0,WOL_05_lect_0005,tari boo xam ne dañu lay gàgganti rekk mokkul
1,WOL_05_lect_0009,dafa am jamano yoo xam ni xasaw dafay ànd ak x...
2,WOL_05_lect_0016,bu ngeen bëggee laale demleen ca àll ba


In [9]:
df_alffa_git.to_csv("DATA\BRUT\DATASET\\alffa_git\\alffa_git_brut.csv")

# Final cleaned data

In [10]:
path_audio = 'DATA\BRUT\DATASET\\alffa_git\\audio\\'
df_alffa_git['length'] = df_alffa_git['filename'].apply(lambda row: resample_audio(row, path_audio))

df_alffa_git.head(3)

,filename,transcript,length
0,WOL_05_lect_0005,tari boo xam ne dañu lay gàgganti rekk mokkul,5.679
1,WOL_05_lect_0009,dafa am jamano yoo xam ni xasaw dafay ànd ak x...,7.095
2,WOL_05_lect_0016,bu ngeen bëggee laale demleen ca àll ba,4.736


In [11]:
df_alffa_git.to_csv("DATA\CLEANED\\alffa_git\\alffa_git_clean.csv")

In [12]:
print(sum(df_alffa_git['length']))

68285.60262499994
